In [1]:
%cd /Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function


In [2]:
import pandas as pd
import M2F

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/venv/lib/python3.12/site-packages/goatools/__init__.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/M2F/dependencies/go-basic.obo: fmt(1.2) rel(2025-07-22) 43,230 Terms


In [3]:
# ENV
from os import getenv
from dotenv import load_dotenv
load_dotenv()

True

# Load and clean the data

In [4]:
df = pd.read_csv(getenv("FETCHED_DATA"))

In [5]:
df.drop(columns=["Protein families", "Rhea ID"], inplace=True)

In [6]:
clean_df = M2F.clean_cols(df,
                          col_names=["Domain [FT]",
                                     "Domain [CC]",
                                     "Gene Ontology (molecular function)",
                                     "Gene Ontology (biological process)",
                                     "Function [CC]",
                                     "Catalytic activity",
                                     "EC number",
                                     "Pathway",
                                     "Cofactor",
                                    "Sequence"],
                          apply_norms={"Domain [FT]" : False,
                                    "Domain [CC]" : True,
                                    "Gene Ontology (molecular function)" : False,
                                    "Gene Ontology (biological process)" : False,
                                    "Function [CC]" : True,
                                    "Catalytic activity" : False,
                                    "EC number" : False,
                                    "Pathway" : True,
                                    "Cofactor" : False,
                                    "Sequence" : False})

# Initialize embedding utils and encode all of the data

In [7]:
cols_of_interest = ["Domain [FT]", "Domain [CC]", "Gene Ontology (molecular function)", "Gene Ontology (biological process)",
 "Function [CC]", "Catalytic activity", "EC number", "Pathway", "Cofactor"]

In [8]:
clean_df_portion = clean_df.head(n=25).copy()

In [9]:
clean_df_portion

,Entry,Domain [FT],Domain [CC],Gene Ontology (molecular function),Gene Ontology (biological process),Function [CC],Catalytic activity,EC number,Pathway,Cofactor,Sequence
0,P77499,"(2..246,)",(),"(GO:0005524, GO:0016887)","(GO:0016226, GO:0009314)",(has low atpase activity the sufbcd complex ac...,(),(),(),(),(MLSIKDLHVSVEDKAILRGLSLDVHPGEVHAIMGPNGSGKSTLSA...
1,P0DP70,"(1..114,)",(),(),"(GO:0015716, GO:0055085)",(c-terminal fragment of the phne protein part ...,(),(),(),(),(MLALFIHTTGVLSKLLSEAVEAIEPGPVEGIRATGANKLEEILYG...
2,Q8A8S8,"(35..222,)",(),"(GO:0033819,)","(GO:0036211,)",(catalyzes the transfer of endogenously produc...,(octanoyl-[ACP] + L-lysyl-[protein] = N(6)-oct...,"(2.3.1.181,)",(protein modification protein lipoylation via ...,(),(MKTVLVDWNLIPYAEAWQRQTEWFDTLVRAKAQGEAYENRIIMCE...
3,P76278,(),(),(),"(GO:0006825,)",(),(),(),(),(),(MLAFTWIALRFIHFTSLMLVFGFAMYGAWLAPLTIRRLLAKRFLR...
4,P77378,(),(),"(GO:0009055, GO:0050660)","(GO:0033539,)","(may play a role in a redox process,)",(),(),(),(),(MSQLNSVWVFSDNPERYAELFGGAQQWGQQVYAIVQNTDQAQAVM...
5,A7ZLQ4,(),(),"(GO:0016862,)",(),(),(),"(5.3.2.-,)",(),(),(MPHIDIKCFPRELDEQQKAALAADITDVIIRHLNSKDSSISIALQ...
6,A0A2D1TVM1,(),(),"(GO:0016757, GO:0015648)","(GO:0051301, GO:0071555, GO:0009252, GO:0008360)",(peptidoglycan polymerase that is essential fo...,([GlcNAc-(1->4)-Mur2Ac(oyl-L-Ala-gamma-D-Glu-L...,"(2.4.99.28,)",(cell wall biogenesis peptidoglycan biosynthes...,(),(MRRPDSDRASSRRSTPRSGRGGQSFSERYIAGVPARIMRPRLIFM...
7,A0A0J1ZC23,"(33..280,)",(),"(GO:0008409, GO:0017108, GO:0003677, GO:000028...","(GO:0033567,)",(has flap endonuclease activity during dna rep...,(),"(3.1.-.-,)",(),"(K(+), Mg(2+))",(MRGLFPISHPAVACSGIECYPYRLIFKGVIVAVHLLIVDALNLIR...
8,W4UUZ6,"(593..667,)",(),"(GO:0003984, GO:1990610, GO:0050660, GO:000028...","(GO:0009097, GO:0009099)",(),"(2 pyruvate + H(+) = (2S)-2-acetolactate + CO2,)","(2.2.1.6,)",(amino-acid biosynthesis l-isoleucine biosynth...,"(Mg(2+), thiamine diphosphate)",(MSSENKTTATSPALPQGEAPQISGAEALMRSLEHQGVKTIFGYPG...
9,A0A174FMU0,"(9..247, 299..442)",(),"(GO:0009011, GO:0004373)","(GO:0005978,)",(synthesizes alpha-14-glucan chains using adp-...,([(1->4)-alpha-D-glucosyl](n) + ADP-alpha-D-gl...,"(2.4.1.21,)","(glycan biosynthesis glycogen biosynthesis,)",(),(MAETNKKMQIVFASAECAPFVKTGGLGDVAGSLPAALVRAGAEVI...


In [10]:
txt_embedder = M2F.FreeTXTEmbedder(getenv("OPENAI_API_KEY"), model="LARGE_OPENAI_MODEL",
                                   cache_file_path="test.db", caching_mode="APPEND")
aa_embedder = M2F.AAChainEmbedder()

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
M2F.embed_ft_domains(clean_df_portion, aa_embedder, inplace=True)
M2F.embed_AAsequences(clean_df_portion, aa_embedder, inplace=True)
M2F.embed_freetxt_cols(clean_df_portion, ["Domain [CC]", "Function [CC]", "Catalytic activity", "Pathway"], txt_embedder, inplace=True)
M2F.encode_go(clean_df_portion, "Gene Ontology (molecular function)", coverage_target=0.9, inplace=True)
M2F.encode_go(clean_df_portion, "Gene Ontology (biological process)", coverage_target=0.9, inplace=True)
M2F.encode_ec(clean_df_portion, "EC number", coverage_target=0.9, inplace=True)
M2F.encode_multihot(clean_df_portion, "Cofactor", inplace=True)
""

''

In [12]:
clean_df_portion

,Entry,Domain [FT],Domain [CC],Gene Ontology (molecular function),Gene Ontology (biological process),Function [CC],Catalytic activity,EC number,Pathway,Cofactor,Sequence
0,P77499,"[-0.21149082, 0.4671508, -0.49195877, 0.645416...",NaN,"(4, 7)","(1,)","[0.023761293, 0.033069983, -0.0029546472, -0.0...",NaN,NaN,NaN,NaN,"[-0.24825583, 0.46281227, -0.5255673, 0.609011..."
1,P0DP70,"[-0.7806524, -0.14354207, 0.20522773, 0.038801...",NaN,NaN,"(1,)","[-0.038915206, 0.029275252, -0.00040166473, 0....",NaN,NaN,NaN,NaN,"[-0.7914472, -0.1744434, 0.2124533, 0.17701457..."
2,Q8A8S8,"[0.3728137, -0.11053239, -0.19788842, 0.049999...",NaN,"(1,)","(1,)","[-0.028294621, 0.009879516, -0.0062410748, -0....","[-0.009099441, 0.0247737, 0.0028672013, 0.0014...","(4,)","[-0.003057203, -0.00897701, 0.0036937413, 0.00...",NaN,"[0.33925, -0.08362906, -0.19572625, 0.08682991..."
3,P76278,NaN,NaN,NaN,"(1,)",NaN,NaN,NaN,NaN,NaN,"[0.10541871, -0.011418763, -0.14937514, 0.1368..."
4,P77378,NaN,NaN,"(1,)","(2, 10)","[-0.026898574, 0.0044630817, -0.0028519693, 0....",NaN,NaN,NaN,NaN,"[0.2664322, -0.1746403, -0.13491875, -0.164157..."
5,A7ZLQ4,NaN,NaN,"(1,)",NaN,NaN,NaN,"(17,)",NaN,NaN,"[0.09223116, -0.15394007, 0.04135332, -0.18273..."
6,A0A2D1TVM1,NaN,NaN,"(1,)","(1,)","[-0.021664396, 0.00047244437, -0.0031731725, 0...","[0.003972352, 0.018654542, -0.0064474847, 0.00...","(6,)","[-0.015242205, 0.031689215, -0.0056352727, 0.0...",NaN,"[0.54539245, -0.032479122, -0.1317679, 0.93089..."
7,A0A0J1ZC23,"[0.13379566, -0.060112715, 0.49114436, 0.06966...",NaN,"(1, 5, 6)","(1,)","[-0.051568206, -9.672521e-05, -0.011024114, -0...",NaN,"(9,)",NaN,"(1, 2)","[0.10255164, -0.10563319, 0.38230208, 0.127237..."
8,W4UUZ6,"[0.13359074, -0.022136545, 0.16895662, -0.5401...",NaN,"(1,)","(3, 12)",NaN,"[0.007461675, 0.012273774, -0.0079097655, -0.0...","(3,)","[-0.028128093, 0.027145084, 0.0044130073, -0.0...","(2, 4)","[0.4225779, -0.12920964, -0.24107683, 0.074338..."
9,A0A174FMU0,"[-1.0119891, -0.85459846, -0.8319432, 0.615693...",NaN,"(2, 3)","(1,)","[0.017259626, 0.028757619, 0.0012161339, -0.00...","[0.0010017961, 0.038359795, -0.0053036264, 0.0...","(5,)","[-0.009604247, 0.056476507, -0.011268787, 0.00...",NaN,"[0.152202, -0.07339708, -0.23733713, 0.5565645..."
